In [1]:
import os
import warnings
import pandas as pd

In [2]:
def loadRes(problem, sizes, methods, new=False, limit60=False):
    # init a list to store processed results
    results = []
    # loop through each size
    for s in sizes:
        for m in methods:
            # retrive metrics
            metrics = retriveData(problem, s, m, new, limit60)
            # append the results to the list
            results.append(metrics)
    # convert into a Pandas DataFrame
    result_df = pd.DataFrame(results)
    # reshape
    result_df = reshapeDataframe(result_df, problem, sizes)
    return result_df

def retriveData(problem, size, method, new, limit60):
    # method map
    method_mapping = {
    "cls": "RC", "thd": "LT", "exact": "EX", 
    "rel": "RR", "root": "N1", "lrn": "RL", "ste": "RS",
    "cls50": "RC", "thd50": "LT", "lrn50": "RL", "ste50": "RS",
    "cls100": "RC", "thd100": "LT", "lrn100": "RL", "ste100": "RS",
    "cls200": "RC", "thd200": "LT", "lrn200": "RL", "ste200": "RS",
    "cls300": "RC", "thd300": "LT", "lrn300": "RL", "ste300": "RS"
    }
    # dir and file
    if limit60 and method in ["exact", "rel", "root"]:
        data_path = "./result60/"
    else:
        data_path = "./result/"
    if problem == "rb":
        if new:
            csv_file = os.path.join(data_path, f"{problem}_{method}_{size}_new.csv")
        else:
            csv_file = os.path.join(data_path, f"{problem}_{method}_{size}.csv")
    else:
        if new:
            csv_file = os.path.join(data_path, f"{problem}_{method}_{size}-{size}_new.csv")
        else:
            csv_file = os.path.join(data_path, f"{problem}_{method}_{size}-{size}.csv")
    #print(csv_file)
    if not os.path.exists(csv_file):
        # output a warning
        warnings.warn(f"File not found: {csv_file}")
        # return None values for all metrics
        return {
            "Method": method_mapping[method],
            "Problem Size": size,
            "Obj Mean": None,
            "Obj Median": None,
            "% Infeasible": None,
            "% Unsolved": None,
            "Time (Sec)": None,
            "Viol Mean": None,
            "Viol Max": None,
            "Num Viol Mean": None
        }
    # load data
    df = pd.read_csv(csv_file)
    # calculate the required metrics
    obj_mean = df["Obj Val"].mean()  # mean of the objective values
    obj_median = df["Obj Val"].median()  # median of the objective values
    if "Num Violations" in df.columns:
        percent_infeasible = (df["Num Violations"] > 0).mean() * 100  # percentage of infeasible instances
    else:
        percent_infeasible = (df["Constraints Viol"] > 0).mean() * 100  # percentage of infeasible instances
    #mean_viols = df["Mean Violation"].mean()  # average mean violation
    #max_viols = df["Max Violation"].max()  # maximum violation
    #num_viols = df["Num Violations"].mean()  # average number of violations
    percent_unsolved = df["Obj Val"].isna().mean() * 100  # percentage of unsolved cases
    time_mean = df["Elapsed Time"].mean()  # Mean of the elapsed solving times
    # append the results to the list
    metrics = {
        "Method": method_mapping[method],
        "Problem Size": size,
        "Obj Mean": obj_mean,
        "Obj Median": obj_median,
        "% Infeasible": percent_infeasible,
        "% Unsolved": percent_unsolved,
        "Time (Sec)": time_mean,
        #"Viol Mean": mean_viols,
        #"Viol Max": max_viols,
        #"Num Viol Mean": num_viols
    }
    return metrics

def reshapeDataframe(df, problem, sizes):
    # define the metrics to include
    metrics = ["Obj Mean", "Obj Median", "% Infeasible", "% Unsolved", "Time (Sec)"]#, "Viol Mean", "Viol Max", "Num Viol Mean"]
    # define the method to include
    methods = ["RC", "LT", "EX", "RR", "N1", "RL", "RS"]
    # reshape
    reshaped_data = []
    for method in methods:
        for metric in metrics:
            row = {"Method": method, "Metric": metric}
            for s in sizes:
                value = df[(df["Method"] == method) & (df["Problem Size"] == s)][metric].values
                # check if the value has exactly one element
                if len(value) != 1:
                    raise ValueError(f"Unexpected number of values ({len(value)}) for Method: {method}, Metric: {metric}, Problem Size: {s}")
                if problem == "rb":
                    row[f"{s*2}×4"] = value[0]
                else:
                    row[f"{s}×{s}"] = value[0]
            reshaped_data.append(row)
    # convert to a DataFrame
    reshaped_df = pd.DataFrame(reshaped_data)
    return reshaped_df

### Convex Quadratic

In [3]:
sizes = [5, 10, 20, 50, 100, 200, 500, 1000]
methods = ["cls", "thd", "exact", "rel", "root", "lrn", "ste"]
loadRes(problem="cq", sizes=sizes, methods=methods)

,Method,Metric,5×5,10×10,20×20,50×50,100×100,200×200,500×500,1000×1000
0,RC,Obj Mean,0.620135,-1.595961e+00,-4.237328e+00,-1.219593e+01,-1.354054e+01,-31.617419,-73.312205,-142.675594
1,RC,Obj Median,0.428045,-1.891001e+00,-4.307350e+00,-1.219962e+01,-1.359898e+01,-31.706233,-73.376950,-142.726241
2,RC,% Infeasible,4.000000,8.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00,3.000000,14.000000,18.000000
3,RC,% Unsolved,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
4,RC,Time (Sec),0.001853,1.865625e-03,1.856163e-03,1.868107e-03,2.159805e-03,0.002132,0.002530,0.004238
5,LT,Obj Mean,0.665490,-1.692566e+00,-4.302138e+00,-1.297686e+01,-1.365074e+01,-31.341756,-72.358103,-142.562616
6,LT,Obj Median,0.467924,-2.014438e+00,-4.319065e+00,-1.302667e+01,-1.377216e+01,-31.613921,-72.483963,-142.552094
7,LT,% Infeasible,2.000000,5.000000e+00,2.000000e+00,2.000000e+00,7.000000e+00,5.000000,6.000000,0.000000
8,LT,% Unsolved,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
9,LT,Time (Sec),0.001943,1.939881e-03,1.973100e-03,1.979415e-03,2.267256e-03,0.002218,0.002589,0.004685


In [4]:
sizes = [5, 10, 20, 50, 100, 200, 500, 1000]
methods = ["cls300", "thd300", "exact", "rel", "root", "lrn300", "ste300"]
loadRes(problem="cq", sizes=sizes, methods=methods)

,Method,Metric,5×5,10×10,20×20,50×50,100×100,200×200,500×500,1000×1000
0,RC,Obj Mean,0.827084,-7.729051e-01,-3.858640e+00,-1.196748e+01,-1.280898e+01,-29.830403,-67.517719,-129.088294
1,RC,Obj Median,0.640985,-1.179969e+00,-3.888334e+00,-1.201319e+01,-1.275984e+01,-29.965430,-67.633426,-129.262868
2,RC,% Infeasible,0.000000,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,2.000000,0.000000,0.000000
3,RC,% Unsolved,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
4,RC,Time (Sec),0.001872,1.932456e-03,2.140489e-03,1.911867e-03,2.107291e-03,0.002469,0.002580,0.004503
5,LT,Obj Mean,0.869226,-1.418925e+00,-3.829681e+00,-1.160599e+01,-1.222682e+01,-30.688119,-69.135814,-120.480242
6,LT,Obj Median,0.671528,-1.702379e+00,-3.818000e+00,-1.173645e+01,-1.229863e+01,-31.029897,-69.221755,-120.651289
7,LT,% Infeasible,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000,0.000000,5.000000
8,LT,% Unsolved,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
9,LT,Time (Sec),0.001889,1.894364e-03,1.885478e-03,1.928422e-03,2.389009e-03,0.002576,0.002771,0.004663


### Simple Non-Convex (Perturbed A)

In [5]:
sizes = [5, 10, 20, 50, 100, 200, 500, 1000]
methods = ["cls100", "thd100", "exact", "rel", "root", "lrn100", "ste100"]
loadRes(problem="nc", sizes=sizes, methods=methods, new=True, limit60=False)

C:\Users\lucas\AppData\Local\Temp\ipykernel_44164\1700917113.py:45: UserWarning: File not found: ./result/nc_exact_20-20_new.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_44164\1700917113.py:45: UserWarning: File not found: ./result/nc_exact_50-50_new.csv
  warnings.warn(f"File not found: {csv_file}")


,Method,Metric,5×5,10×10,20×20,50×50,100×100,200×200,500×500,1000×1000
0,RC,Obj Mean,0.382745,1.101563,0.227695,7.712910e-01,1.663565,1.471528,0.526173,1.422244
1,RC,Obj Median,0.239879,0.837723,0.216590,7.519165e-01,1.594214,1.436150,0.526173,0.809124
2,RC,% Infeasible,1.000000,3.000000,0.000000,2.000000e+00,0.000000,1.000000,4.000000,3.000000
3,RC,% Unsolved,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
4,RC,Time (Sec),0.001919,0.001936,0.001928,2.014108e-03,0.002193,0.002170,0.002871,0.003962
5,LT,Obj Mean,0.359111,0.909994,0.194875,5.796574e-01,0.668875,-0.356054,-1.373667,-3.743579
6,LT,Obj Median,0.225594,0.682508,0.175219,5.663435e-01,0.648688,-0.373353,-1.593921,-3.716251
7,LT,% Infeasible,0.000000,2.000000,1.000000,2.000000e+00,4.000000,0.000000,2.000000,1.000000
8,LT,% Unsolved,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
9,LT,Time (Sec),0.001949,0.001943,0.001912,1.978652e-03,0.002149,0.002340,0.002877,0.004997


In [6]:
sizes = [5, 10, 20, 50, 100, 200, 500, 1000]
methods = ["cls300", "thd300", "exact", "rel", "root", "lrn300", "ste300"]
loadRes(problem="nc", sizes=sizes, methods=methods, new=True, limit60=False)

C:\Users\lucas\AppData\Local\Temp\ipykernel_44164\1700917113.py:45: UserWarning: File not found: ./result/nc_exact_20-20_new.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_44164\1700917113.py:45: UserWarning: File not found: ./result/nc_exact_50-50_new.csv
  warnings.warn(f"File not found: {csv_file}")


,Method,Metric,5×5,10×10,20×20,50×50,100×100,200×200,500×500,1000×1000
0,RC,Obj Mean,0.854757,1.236293,0.597644,1.351623e+00,1.207939,1.521782,-0.350723,4.050655
1,RC,Obj Median,0.495905,0.865282,0.601463,1.325490e+00,1.197777,1.521782,-0.340753,3.868986
2,RC,% Infeasible,1.000000,0.000000,0.000000,0.000000e+00,1.000000,0.000000,1.000000,0.000000
3,RC,% Unsolved,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
4,RC,Time (Sec),0.001909,0.001911,0.001915,2.013597e-03,0.002084,0.002171,0.002737,0.004916
5,LT,Obj Mean,0.777542,1.175287,0.607586,9.323977e-01,1.626526,0.728979,0.254451,6.651209
6,LT,Obj Median,0.460508,0.786141,0.600822,9.162173e-01,1.619405,0.699982,0.172843,6.368074
7,LT,% Infeasible,0.000000,1.000000,0.000000,0.000000e+00,3.000000,0.000000,0.000000,5.000000
8,LT,% Unsolved,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
9,LT,Time (Sec),0.001930,0.001921,0.001935,1.958737e-03,0.002217,0.002173,0.002600,0.005406


### Rosenbrock

In [7]:
sizes = [1, 10, 100, 1000, 10000]
methods = ["cls100", "thd100", "exact", "rel", "root", "lrn100", "ste100"]
loadRes(problem="rb", sizes=sizes, methods=methods, limit60=False)

C:\Users\lucas\AppData\Local\Temp\ipykernel_44164\1700917113.py:45: UserWarning: File not found: ./result/rb_exact_1000.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_44164\1700917113.py:45: UserWarning: File not found: ./result/rb_rel_10000.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_44164\1700917113.py:45: UserWarning: File not found: ./result/rb_root_10000.csv
  warnings.warn(f"File not found: {csv_file}")


,Method,Metric,2×4,20×4,200×4,2000×4,20000×4
0,RC,Obj Mean,23.265538,59.393365,5.035068e+02,5.938372e+03,6.688320e+04
1,RC,Obj Median,21.475520,48.857842,4.617061e+02,5.792519e+03,6.679711e+04
2,RC,% Infeasible,3.000000,0.000000,1.000000e+00,1.000000e+00,2.400000e+01
3,RC,% Unsolved,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
4,RC,Time (Sec),0.001926,0.001944,2.087076e-03,3.257055e-03,1.213258e-02
5,LT,Obj Mean,23.182455,62.508514,6.227751e+02,5.611775e+03,4.762218e+04
6,LT,Obj Median,20.796687,63.404079,6.260387e+02,5.557821e+03,3.451887e+04
7,LT,% Infeasible,2.000000,0.000000,0.000000e+00,3.000000e+00,3.400000e+01
8,LT,% Unsolved,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
9,LT,Time (Sec),0.001930,0.001988,2.615485e-03,2.994518e-03,1.272431e-02


In [8]:
sizes = [1, 10, 100, 1000, 10000]
#methods = ["cls100", "thd100", "exact", "rel", "root", "lrn100", "ste100"]
#methods = ["cls200", "thd200", "exact", "rel", "root", "lrn200", "ste200"]
methods = ["cls300", "thd300", "exact", "rel", "root", "lrn300", "ste300"]
loadRes(problem="rb", sizes=sizes, methods=methods, limit60=False)

C:\Users\lucas\AppData\Local\Temp\ipykernel_44164\1700917113.py:45: UserWarning: File not found: ./result/rb_exact_1000.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_44164\1700917113.py:45: UserWarning: File not found: ./result/rb_rel_10000.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_44164\1700917113.py:45: UserWarning: File not found: ./result/rb_root_10000.csv
  warnings.warn(f"File not found: {csv_file}")


,Method,Metric,2×4,20×4,200×4,2000×4,20000×4
0,RC,Obj Mean,23.844495,53.938966,5.509479e+02,6.034635e+03,6.046596e+04
1,RC,Obj Median,21.774059,50.482929,5.361845e+02,5.864990e+03,5.071739e+04
2,RC,% Infeasible,0.000000,0.000000,0.000000e+00,0.000000e+00,3.100000e+01
3,RC,% Unsolved,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
4,RC,Time (Sec),0.001899,0.001896,2.167630e-03,3.227007e-03,1.259692e-02
5,LT,Obj Mean,23.397848,54.967154,5.636693e+02,5.465797e+03,1.859685e+05
6,LT,Obj Median,21.004716,52.338176,5.418099e+02,5.400318e+03,1.745040e+05
7,LT,% Infeasible,0.000000,0.000000,0.000000e+00,0.000000e+00,4.500000e+01
8,LT,% Unsolved,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
9,LT,Time (Sec),0.001952,0.001939,2.183278e-03,3.128786e-03,1.190448e-02
